In [ ]:
!pip install beautifulsoup4 
!pip install emoji 
!pip install transformers

     |████████████████████████████████| 175 kB 4.4 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=bccb91bbcbd9dd93898aaf0b223b5ff1a0b2551e2652ce577491b7f08d60a13a
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji
     |████████████████████████████████| 4.2 MB 4.3 MB/s 
     |████████████████████████████████| 596 kB 37.4 MB/s 
     |████████████████████████████████| 84 kB 3.4 MB/s 
     |████████████████████████████████| 6.6 MB 22.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import re
import csv
import tqdm
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from time import process_time
from joblib import dump
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models.ldamodel import LdaModel
import tensorflow as tf
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Layer
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import plot_model
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Embedding, Input, Dense, LSTM, Dropout, Flatten, Bidirectional, Conv1D, concatenate, SpatialDropout1D
from tensorflow.keras.layers import MaxPooling1D, MaxPooling2D, GlobalMaxPooling1D, GlobalAveragePooling2D, GlobalAveragePooling1D 
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler, EarlyStopping
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')

stopwords = stopwords.words('english')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from keras.utils import np_utils
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import tensorflow as tf
from keras import layers
from keras.models import Sequential

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import re
import string
import json
import emoji
import numpy as np
import pandas as pd
from sklearn import metrics
from bs4 import BeautifulSoup
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, AutoTokenizer, BertModel, BertConfig, AutoModel, AdamW
import warnings
warnings.filterwarnings('ignore')

pd.set_option("display.max_columns", None)

In [ ]:
df_train = pd.read_csv("drive/MyDrive/train.tsv", sep='\t', header=None, names=['Text', 'Class', 'ID'])
df_dev = pd.read_csv("drive/MyDrive/dev.tsv", sep='\t', header=None, names=['Text', 'Class', 'ID'])
df_test = pd.read_csv("drive/MyDrive/test.tsv", sep='\t', header=None, names=['Text', 'Class', 'ID'])

In [ ]:
df_train

,Text,Class,ID
0,My favourite food is anything I didn't have to...,27,eebbqej
1,"Now if he does off himself, everyone will thin...",27,ed00q6i
2,WHY THE FUCK IS BAYLESS ISOING,2,eezlygj
3,To make her feel threatened,14,ed7ypvh
4,Dirty Southern Wankers,3,ed0bdzj
...,...,...,...
43405,Added you mate well I’ve just got the bow and ...,18,edsb738
43406,Always thought that was funny but is it a refe...,6,ee7fdou
43407,What are you talking about? Anything bad that ...,3,efgbhks
43408,"More like a baptism, with sexy results!",13,ed1naf8


# New Section

In [ ]:
df_train['List of classes'] = df_train['Class'].apply(lambda x: x.split(','))
df_train['Len of classes'] = df_train['List of classes'].apply(lambda x: len(x))
df_dev['List of classes'] = df_dev['Class'].apply(lambda x: x.split(','))
df_dev['Len of classes'] = df_dev['List of classes'].apply(lambda x: len(x))

In [ ]:
df_train.head()

,Text,Class,ID,List of classes,Len of classes
0,My favourite food is anything I didn't have to...,27,eebbqej,[27],1
1,"Now if he does off himself, everyone will thin...",27,ed00q6i,[27],1
2,WHY THE FUCK IS BAYLESS ISOING,2,eezlygj,[2],1
3,To make her feel threatened,14,ed7ypvh,[14],1
4,Dirty Southern Wankers,3,ed0bdzj,[3],1


In [ ]:
df_test['List of classes'] = df_test['Class'].apply(lambda x: x.split(','))
df_test['Len of classes'] = df_test['List of classes'].apply(lambda x: len(x))

In [ ]:

with open('drive/MyDrive/ekman_mapping.json') as file:
    ekman_mapping = json.load(file)

In [ ]:
emotion_file = open("drive/MyDrive/emotions.txt", "r")
emotion_list = emotion_file.read()
emotion_list = emotion_list.split("\n")
print(emotion_list)

['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral']


In [ ]:
def idx2class(idx_list):
    arr = []
    for i in idx_list:
        arr.append(emotion_list[int(i)])
    return arr

In [ ]:
df_train['Emotions'] = df_train['List of classes'].apply(idx2class)
df_dev['Emotions'] = df_dev['List of classes'].apply(idx2class)
df_test['Emotions'] = df_test['List of classes'].apply(idx2class)

In [ ]:
df_train.head()

,Text,Class,ID,List of classes,Len of classes,Emotions
0,My favourite food is anything I didn't have to...,27,eebbqej,[27],1,[neutral]
1,"Now if he does off himself, everyone will thin...",27,ed00q6i,[27],1,[neutral]
2,WHY THE FUCK IS BAYLESS ISOING,2,eezlygj,[2],1,[anger]
3,To make her feel threatened,14,ed7ypvh,[14],1,[fear]
4,Dirty Southern Wankers,3,ed0bdzj,[3],1,[annoyance]


In [ ]:
def EmotionMapping(emotion_list):
    map_list = []
    
    for i in emotion_list:
        if i in ekman_mapping['anger']:
            map_list.append('anger')
        if i in ekman_mapping['disgust']:
            map_list.append('disgust')
        if i in ekman_mapping['fear']:
            map_list.append('fear')
        if i in ekman_mapping['joy']:
            map_list.append('joy')
        if i in ekman_mapping['sadness']:
            map_list.append('sadness')
        if i in ekman_mapping['surprise']:
            map_list.append('surprise')
        if i == 'neutral':
            map_list.append('neutral')
            
    return map_list

In [ ]:
df_train['Mapped Emotions'] = df_train['Emotions'].apply(EmotionMapping)
df_dev['Mapped Emotions'] = df_dev['Emotions'].apply(EmotionMapping)

In [ ]:
df_test['Mapped Emotions'] = df_test['Emotions'].apply(EmotionMapping)

In [ ]:
df_train['anger'] = np.zeros((len(df_train),1))
df_train['disgust'] = np.zeros((len(df_train),1))
df_train['fear'] = np.zeros((len(df_train),1))
df_train['joy'] = np.zeros((len(df_train),1))
df_train['sadness'] = np.zeros((len(df_train),1))
df_train['surprise'] = np.zeros((len(df_train),1))
df_train['neutral'] = np.zeros((len(df_train),1))

df_dev['anger'] = np.zeros((len(df_dev),1))
df_dev['disgust'] = np.zeros((len(df_dev),1))
df_dev['fear'] = np.zeros((len(df_dev),1))
df_dev['joy'] = np.zeros((len(df_dev),1))
df_dev['sadness'] = np.zeros((len(df_dev),1))
df_dev['surprise'] = np.zeros((len(df_dev),1))
df_dev['neutral'] = np.zeros((len(df_dev),1))

In [ ]:
df_test['anger'] = np.zeros((len(df_test),1))
df_test['disgust'] = np.zeros((len(df_test),1))
df_test['fear'] = np.zeros((len(df_test),1))
df_test['joy'] = np.zeros((len(df_test),1))
df_test['sadness'] = np.zeros((len(df_test),1))
df_test['surprise'] = np.zeros((len(df_test),1))
df_test['neutral'] = np.zeros((len(df_test),1))

In [ ]:
for i in ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise','neutral']:
    df_train[i] = df_train['Mapped Emotions'].apply(lambda x: 1 if i in x else 0)
    df_dev[i] = df_dev['Mapped Emotions'].apply(lambda x: 1 if i in x else 0)

In [ ]:
for i in ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise','neutral']:
    df_test[i] = df_test['Mapped Emotions'].apply(lambda x: 1 if i in x else 0)

In [ ]:
df_train.head()

,Text,Class,ID,List of classes,Len of classes,Emotions,Mapped Emotions,anger,disgust,fear,joy,sadness,surprise,neutral
0,My favourite food is anything I didn't have to...,27,eebbqej,[27],1,[neutral],[neutral],0,0,0,0,0,0,1
1,"Now if he does off himself, everyone will thin...",27,ed00q6i,[27],1,[neutral],[neutral],0,0,0,0,0,0,1
2,WHY THE FUCK IS BAYLESS ISOING,2,eezlygj,[2],1,[anger],[anger],1,0,0,0,0,0,0
3,To make her feel threatened,14,ed7ypvh,[14],1,[fear],[fear],0,0,1,0,0,0,0
4,Dirty Southern Wankers,3,ed0bdzj,[3],1,[annoyance],[anger],1,0,0,0,0,0,0


In [ ]:
df_dev['Mapped Emotions'].value_counts

<bound method IndexOpsMixin.value_counts of 0            [neutral]
1       [joy, neutral]
2            [sadness]
3       [joy, neutral]
4                [joy]
             ...      
5421            [fear]
5422             [joy]
5423           [anger]
5424           [anger]
5425             [joy]
Name: Mapped Emotions, Length: 5426, dtype: object>

In [ ]:
df_train.drop(df_train[df_train['neutral'] == 1].index, inplace=True)
df_dev.drop(df_dev[df_dev['neutral'] == 1].index, inplace=True)
df_train.drop(df_train[df_train['disgust'] == 1].index, inplace=True)
df_dev.drop(df_dev[df_dev['disgust'] == 1].index, inplace=True)

In [ ]:


df_test.drop(df_test[df_test['neutral'] == 1].index, inplace=True)
df_test.drop(df_test[df_test['disgust'] == 1].index, inplace=True)

In [ ]:
df_train.drop(['Class', 'List of classes', 'Len of classes', 'Emotions', 'Mapped Emotions'], axis=1, inplace=True)#'neutral', 'disgust'
df_dev.drop(['Class', 'List of classes', 'Len of classes', 'Emotions', 'Mapped Emotions'], axis=1, inplace=True)

In [ ]:
df_test.drop(['Class', 'List of classes', 'Len of classes', 'Emotions', 'Mapped Emotions'], axis=1, inplace=True)

In [ ]:
contraction_mapping = { "ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", 
                       "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", 
                       "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", 
                       "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am",
                       "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", 
                       "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have",
                       "it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not",
                       "mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", 
                       "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not",
                       "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", 
                       "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have",
                       "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is",
                       "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would",
                       "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have",
                       "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have",
                       "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", 
                       "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did",
                       "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", 
                       "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", 
                       "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
                       "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have",
                       "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have", 'u.s':'america', 'e.g':'for example'}

punct = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-",
                 "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 
                 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', '!':' '}

mispell_dict = {'colour': 'color', 'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling', 'counselling': 'counseling', 'theatre': 'theater',
                'cancelled': 'canceled', 'labour': 'labor', 'organisation': 'organization', 'wwii': 'world war 2', 'citicise': 'criticize', 'youtu ': 'youtube ',
                'Qoura': 'Quora', 'sallary': 'salary', 'Whta': 'What', 'narcisist': 'narcissist', 'howdo': 'how do', 'whatare': 'what are', 'howcan': 'how can',
                'howmuch': 'how much', 'howmany': 'how many', 'whydo': 'why do', 'doI': 'do I', 'theBest': 'the best', 'howdoes': 'how does', 
                'mastrubation': 'masturbation', 'mastrubate': 'masturbate', "mastrubating": 'masturbating', 'pennis': 'penis', 'Etherium': 'Ethereum', 
                'narcissit': 'narcissist', 'bigdata': 'big data', '2k17': '2017', '2k18': '2018', 'qouta': 'quota', 'exboyfriend': 'ex boyfriend', 
                'airhostess': 'air hostess', "whst": 'what', 'watsapp': 'whatsapp', 'demonitisation': 'demonetization', 'demonitization': 'demonetization',
                'demonetisation': 'demonetization'}

In [ ]:
def clean_text(text):
    '''Clean emoji, Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = emoji.demojize(text)
    text = re.sub(r'\:(.*?)\:','',text)
    text = str(text).lower()    #Making Text Lowercase
    text = re.sub('\[.*?\]', '', text)
    #The next 2 lines remove html text
    text = BeautifulSoup(text, 'lxml').get_text()
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",", "'")
    text = re.sub(r"[^a-zA-Z?.!,¿']+", " ", text)
    return text

def clean_contractions(text, mapping):
    '''Clean contraction using contraction mapping'''    
    specials = ["’", "‘", "´", "`"]
    for s in specials:
        text = text.replace(s, "'")
    for word in mapping.keys():
        if ""+word+"" in text:
            text = text.replace(""+word+"", ""+mapping[word]+"")
    #Remove Punctuations
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    text = re.sub(r"([?.!,¿])", r" \1 ", text)
    text = re.sub(r'[" "]+', " ", text)
    return text

def clean_special_chars(text, punct, mapping):
    '''Cleans special characters present(if any)'''   
    for p in mapping:
        text = text.replace(p, mapping[p])
    
    for p in punct:
        text = text.replace(p, f' {p} ')
    
    specials = {'\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': ''}  
    for s in specials:
        text = text.replace(s, specials[s])
    
    return text

def correct_spelling(x, dic):
    '''Corrects common spelling errors'''   
    for word in dic.keys():
        x = x.replace(word, dic[word])
    return x

def remove_space(text):
    '''Removes  spaces'''   
    #Removes  spaces 
    text = text.strip()
    text = text.split()
    return " ".join(text)

def text_preprocessing_pipeline(text):
    '''Cleaning and parsing the text.'''
    text = clean_text(text)
    text = clean_contractions(text, contraction_mapping)
    text = clean_special_chars(text, punct, punct_mapping)
    text = correct_spelling(text, mispell_dict)
    text = remove_space(text)
    return text

In [ ]:
def fun(s):
    z=""
    for word in s:
      if word in dct:
        z=z+dct[word]+" "
      else:
        z=z+word
    return z

In [ ]:
# df_train['new_text'] = df_train['Text'].apply(fun)
# df_dev['new_text'] = df_dev['Text'].apply(fun)
# df_test['new_text'] = df_test['Text'].apply(fun)

In [ ]:
df_train['Text'] = df_train['Text'].apply(text_preprocessing_pipeline)
df_dev['Text'] = df_dev['Text'].apply(text_preprocessing_pipeline)
df_test['Text'] = df_test['Text'].apply(text_preprocessing_pipeline)

In [ ]:
# df_train['new_text'] = df_train['new_text'].apply(text_preprocessing_pipeline)
# df_dev['new_text'] = df_dev['new_text'].apply(text_preprocessing_pipeline)
# df_test['new_text'] = df_test['new_text'].apply(text_preprocessing_pipeline)

In [ ]:
df_train.reset_index(drop=True).to_csv("train.csv", index=False)
df_dev.reset_index(drop=True).to_csv("val.csv", index=False)
df_test.reset_index(drop=True).to_csv("test.csv", index=False)

In [ ]:
df_train = df_train.reset_index(drop=True)
df_dev = df_dev.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [ ]:
df_train.head()

,Text,ID,anger,disgust,fear,joy,sadness,surprise,neutral
0,my favorite food is anything i did not have to...,eebbqej,0,0,0,0,0,0,1
1,now if he does off himself everyone will think...,ed00q6i,0,0,0,0,0,0,1
2,why the fuck is bayless isoing,eezlygj,1,0,0,0,0,0,0
3,to make her feel threatened,ed7ypvh,0,0,1,0,0,0,0
4,dirty southern wankers,ed0bdzj,1,0,0,0,0,0,0


In [ ]:
print(df_train.shape)
print(df_dev.shape)

(43410, 9)
(5426, 9)


In [ ]:
df_train.columns

Index(['Text', 'ID', 'anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise',
       'neutral'],
      dtype='object')

In [ ]:
df_dev.columns

Index(['Text', 'ID', 'anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise',
       'neutral'],
      dtype='object')

In [ ]:
# tot=0
# for i in ['anger', 'fear', 'joy', 'sadness', 'surprise']:
#   z=df_train[i].sum()
#   tot=tot+z;
#   print(i+" "+str(z))

In [ ]:
X_train = df_train['Text']
X_test = df_dev['Text']
y_train = df_train.iloc[:, 2:9]
y_test = df_dev.iloc[:, 2:9]
X_t = df_test['Text']
y_t = df_test.iloc[:, 2:9]

In [ ]:
# for index, row in df_train.iterrows():
#     z=0
#     z=row['anger']+row['fear']+row['sadness']+row['surprise']+row['joy']
#     if z>1 :
#       print(row['Text'])

In [ ]:
sentences_ted = []
for sent_str in X_train:
    tokens = re.sub(r"[^a-z0-9]+", " ", sent_str.lower()).split()
    sentences_ted.append(tokens)
for sent_str in X_test:
    tokens = re.sub(r"[^a-z0-9]+", " ", sent_str.lower()).split()
    sentences_ted.append(tokens)

In [ ]:
from keras.utils import np_utils
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import tensorflow as tf
from keras import layers
from keras.models import Sequential

EPOCHS= 30
act= "swish"
opt= tf.keras.optimizers.Adam(learning_rate=0.001)

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
X_t = tokenizer.texts_to_sequences(X_t)
# #TEST = tokenizer.texts_to_sequences(test_df["clean_text"]) 

vocab_size = len(tokenizer.word_index) + 1

In [ ]:
lens_train = [len(i) for i in X_train]
lens_test = [len(i) for i in X_test]
lens = lens_train + lens_test

maxlen = np.max(lens)
print('Max len:', maxlen)
maxlen=40

Max len: 40


In [ ]:
# lens_test = [len(i) for i in X_t]
# lens = lens_test
# maxlen = np.max(lens)
# print('Max len:', maxlen)
# print(lens)

In [ ]:
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)
X_t = pad_sequences(X_t, padding='post', maxlen=maxlen)

In [ ]:
sentences_ted

[['my',
  'favorite',
  'food',
  'is',
  'anything',
  'i',
  'did',
  'not',
  'have',
  'to',
  'cook',
  'myself'],
 ['now',
  'if',
  'he',
  'does',
  'off',
  'himself',
  'everyone',
  'will',
  'think',
  'hes',
  'having',
  'a',
  'laugh',
  'screwing',
  'with',
  'people',
  'instead',
  'of',
  'actually',
  'dead'],
 ['why', 'the', 'fuck', 'is', 'bayless', 'isoing'],
 ['to', 'make', 'her', 'feel', 'threatened'],
 ['dirty', 'southern', 'wankers'],
 ['omg',
  'peyton',
  'is',
  'not',
  'good',
  'enough',
  'to',
  'help',
  'us',
  'in',
  'the',
  'playoffs',
  'dumbass',
  'broncos',
  'fans',
  'circa',
  'december'],
 ['yes',
  'i',
  'heard',
  'abt',
  'the',
  'f',
  'bombs',
  'that',
  'has',
  'to',
  'be',
  'why',
  'thanks',
  'for',
  'your',
  'replysquinting',
  'face',
  'with',
  'tongue'],
 ['we',
  'need',
  'more',
  'boards',
  'and',
  'to',
  'create',
  'a',
  'bit',
  'more',
  'space',
  'for',
  'then',
  'we',
  'll',
  'be',
  'good'],
 ['d

In [ ]:
from gensim.models import FastText
model_ted = FastText(sentences_ted, size=300, window=5, min_count=5, workers=4,sg=1)

In [ ]:
word_index = tokenizer.word_index
embedding_matrix3 = np.random.random((vocab_size, 300))
for word,i in word_index.items():
    try:
        embedding_vector = model_ted.wv[word]
    except:
        print(word, 'not found')
    if embedding_vector is not None:
        embedding_matrix3[i] = embedding_vector

gc not found
nz not found
bj not found
cgi not found
dvd not found
pkk not found
uwu not found
pk not found
db not found
dv not found
fkn not found
oz not found
hp not found
pnw not found
uj not found
dw not found
xoxo not found
fwiw not found
gdt not found
qp not found
mnf not found
mf not found
bmw not found
pbr not found
ttk not found
bjj not found
wb not found
iw not found
lg not found
pnr not found
xo not found
xl not found
kd not found
cv not found
2 not found
lhw not found
kkk not found
lgd not found
ln not found
lw not found
pg not found
uyu not found
xx not found
pyd not found
pb not found
hq not found
cd not found
nm not found
pdx not found
fk not found
svu not found
kmt not found
oj not found
sv not found
nl not found
bnw not found
fn not found
hbk not found
iy not found
kt not found
hv not found
gx not found
tgr not found
hb not found
ksg not found
pj not found
hpv not found
vb not found
cdjr not found
uc not found
jpg not found
lsu not found
hf not found
rfm not found
nw n

In [ ]:
embedding_matrix=embedding_matrix3
embedding_matrix.shape[0]

26155

In [ ]:
embedding_dim = 300

model = Sequential()
model.add(layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, weights = [embedding_matrix], input_length=maxlen))
model.add(layers.Dropout(0.3)) 
model.add(layers.Conv1D(filters=32, kernel_size=3, activation=act)) 
model.add(layers.MaxPool1D(pool_size=3)) 
model.add(layers.Dropout(0.3))
model.add(layers.Conv1D(filters=32, kernel_size=3, activation=act)) 
model.add(layers.MaxPool1D(pool_size=3)) 
model.add(layers.Dropout(0.3))
model.add(layers.Bidirectional(layers.LSTM(256, recurrent_dropout=0.3)))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(256,activation=act)) 
model.add(layers.Dropout(0.3)) 
model.add(layers.Dense(7, activation="sigmoid"))
model.compile(optimizer=opt, loss="binary_crossentropy",metrics=["accuracy"])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 40, 300)           7846500   
                                                                 
 dropout_5 (Dropout)         (None, 40, 300)           0         
                                                                 
 conv1d_2 (Conv1D)           (None, 38, 32)            28832     
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 12, 32)           0         
 1D)                                                             
                                                                 
 dropout_6 (Dropout)         (None, 12, 32)            0         
                                                                 
 conv1d_3 (Conv1D)           (None, 10, 32)            3104      
                                                      

In [ ]:
Input1 = Input(shape=(maxlen,))
input_dim1=embedding_matrix.shape[0]
embedding_dim1=300
Embedding_layer1 = Embedding(input_dim=input_dim1, 
                             output_dim=embedding_dim1, 
                             weights=[embedding_matrix], 
                             trainable=False)(Input1)

In [ ]:
Lstm = Bidirectional(LSTM(64, dropout=0.3, return_sequences=True))(Embedding_layer1)
Lstm = Flatten()(Lstm)
# Lstm = Dense(64,activation='relu')(Lstm)
convs = []
filter_sizes = [3,4,5]
for filter_size in filter_sizes:
    l_conv = Conv1D(filters=200, 
                    kernel_size=filter_size, 
                    activation='relu')(Embedding_layer1)
    l_pool = GlobalMaxPooling1D()(l_conv)
    convs.append(l_pool)
l_merge = concatenate(convs, axis=1)
# l_merge = Dense(64,activation='relu')(l_merge)
conc = concatenate([Lstm, l_merge], axis=1)
Dense1 = Dense(32, activation='relu')(conc)
Dropout_dense = Dropout(0.4)(Dense1)
outputs = Dense(5, activation='sigmoid')(Dropout_dense)
classifier = Model(inputs=Input1, outputs=outputs)
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
classifier.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 40)]         0           []                               
                                                                                                  
 embedding_3 (Embedding)        (None, 40, 300)      5994000     ['input_1[0][0]']                
                                                                                                  
 conv1d_6 (Conv1D)              (None, 38, 200)      180200      ['embedding_3[0][0]']            
                                                                                                  
 conv1d_7 (Conv1D)              (None, 37, 200)      240200      ['embedding_3[0][0]']            
                                                                                              

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler, EarlyStopping
checkpoint_filepath = "model_7emotion.h5"
model_checkpoint_callback = ModelCheckpoint(filepath=checkpoint_filepath,
                                            save_weights_only=True,
                                            monitor='val_accuracy',
                                            mode='max',
                                            save_freq = 'epoch',
                                            save_best_only=True)

reduce_lr_callback = ReduceLROnPlateau( monitor='val_accuracy', 
                                       factor=0.1, 
                                       patience=2,                                  
                                       verbose=1)

In [ ]:
history = model.fit(X_train, y_train, epochs=EPOCHS, verbose=1, validation_data=(X_test, y_test), batch_size=64, callbacks=[model_checkpoint_callback,reduce_lr_callback]) 

Epoch 1/30
679/679 [==============================] - 124s 174ms/step - loss: 0.3483 - accuracy: 0.4224 - val_loss: 0.2954 - val_accuracy: 0.5614 - lr: 0.0010
Epoch 2/30
679/679 [==============================] - 114s 168ms/step - loss: 0.2876 - accuracy: 0.5732 - val_loss: 0.2728 - val_accuracy: 0.5953 - lr: 0.0010
Epoch 3/30
679/679 [==============================] - 115s 170ms/step - loss: 0.2615 - accuracy: 0.6209 - val_loss: 0.2726 - val_accuracy: 0.5957 - lr: 0.0010
Epoch 4/30
679/679 [==============================] - 117s 172ms/step - loss: 0.2397 - accuracy: 0.6601 - val_loss: 0.2750 - val_accuracy: 0.6050 - lr: 0.0010
Epoch 5/30
679/679 [==============================] - 114s 167ms/step - loss: 0.2192 - accuracy: 0.6979 - val_loss: 0.2749 - val_accuracy: 0.6025 - lr: 0.0010
Epoch 6/30
679/679 [==============================] - ETA: 0s - loss: 0.2002 - accuracy: 0.7297
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
679/679 [=======================

In [ ]:
# loss, accuracy = classifier.evaluate(X_t, y_t, verbose=0)

In [ ]:
model_curr=model

In [ ]:
import torch
model = torch.load('drive/MyDrive/model_13may.pt')

AttributeError: ignored

In [ ]:
model_curr.load_weights('drive/MyDrive/model_24.h5')

In [ ]:
model_curr.save("NewH5model.h5")

In [ ]:
import joblib

joblib.dump(model_curr, 'Wmodel.pkl')

INFO:tensorflow:Assets written to: ram://0e36a1cb-0117-45da-a33b-6702eb512ad6/assets


['Wmodel.pkl']

In [ ]:
joblib.dump(tokenizer, 'tokenizer.pkl')

['tokenizer.pkl']

In [ ]:
curr_model = joblib.load('Wmodel.pkl')

In [ ]:
loss, accuracy = model_curr.evaluate(X_t, y_t, verbose=0)

In [ ]:
accuracy

0.7181817889213562

In [ ]:
# arr = model.predict(X_t[4])

In [ ]:
# model1.load_weights('advkkmodel.h5')

In [ ]:
# model.save_weights('goodkkkmodel.h5')

In [ ]:
# def fun(sen):
#   arr = model.predict(z)
#   lt=[]
#   k=0
#   ind=0
#   mx=0
#   for i in arr[0]:
#     # print(i)
#     if i>0.5:
#       lt.append(dct[k])
#     if i>mx:
#       mx=i
#       ind=k
#     k=k+1
#   if len(lt)==0:
#   lt.append(dct[ind])
  

In [ ]:
df_test['Text'][208]

'sounds good'

In [ ]:
import io
dct={}
dct[0]='anger'
dct[1]='fear'
dct[2]='joy'
dct[3]='sadness'
dct[4]='surprise'

tokenizer1 = tokenizer

sen='OMG, yep!!! that is final answer! Thank you so much'


contraction_mapping = { "ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", 
                       "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", 
                       "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", 
                       "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am",
                       "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", 
                       "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have",
                       "it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not",
                       "mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", 
                       "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not",
                       "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", 
                       "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have",
                       "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is",
                       "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would",
                       "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have",
                       "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have",
                       "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", 
                       "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did",
                       "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", 
                       "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", 
                       "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
                       "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have",
                       "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have", 'u.s':'america', 'e.g':'for example'}

punct = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-",
                 "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 
                 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', '!':' '}

mispell_dict = {'colour': 'color', 'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling', 'counselling': 'counseling', 'theatre': 'theater',
                'cancelled': 'canceled', 'labour': 'labor', 'organisation': 'organization', 'wwii': 'world war 2', 'citicise': 'criticize', 'youtu ': 'youtube ',
                'Qoura': 'Quora', 'sallary': 'salary', 'Whta': 'What', 'narcisist': 'narcissist', 'howdo': 'how do', 'whatare': 'what are', 'howcan': 'how can',
                'howmuch': 'how much', 'howmany': 'how many', 'whydo': 'why do', 'doI': 'do I', 'theBest': 'the best', 'howdoes': 'how does', 
                'mastrubation': 'masturbation', 'mastrubate': 'masturbate', "mastrubating": 'masturbating', 'pennis': 'penis', 'Etherium': 'Ethereum', 
                'narcissit': 'narcissist', 'bigdata': 'big data', '2k17': '2017', '2k18': '2018', 'qouta': 'quota', 'exboyfriend': 'ex boyfriend', 
                'airhostess': 'air hostess', "whst": 'what', 'watsapp': 'whatsapp', 'demonitisation': 'demonetization', 'demonitization': 'demonetization',
                'demonetisation': 'demonetization'}


def clean_text(text):
    '''Clean emoji, Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = emoji.demojize(text)
    text = re.sub(r'\:(.*?)\:','',text)
    text = str(text).lower()    #Making Text Lowercase
    text = re.sub('\[.*?\]', '', text)
    #The next 2 lines remove html text
    text = BeautifulSoup(text, 'lxml').get_text()
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",", "'")
    text = re.sub(r"[^a-zA-Z?.!,¿']+", " ", text)
    return text

def clean_contractions(text, mapping):
    '''Clean contraction using contraction mapping'''    
    specials = ["’", "‘", "´", "`"]
    for s in specials:
        text = text.replace(s, "'")
    for word in mapping.keys():
        if ""+word+"" in text:
            text = text.replace(""+word+"", ""+mapping[word]+"")
    #Remove Punctuations
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    text = re.sub(r"([?.!,¿])", r" \1 ", text)
    text = re.sub(r'[" "]+', " ", text)
    return text

def clean_special_chars(text, punct, mapping):
    '''Cleans special characters present(if any)'''   
    for p in mapping:
        text = text.replace(p, mapping[p])
    
    for p in punct:
        text = text.replace(p, f' {p} ')
    
    specials = {'\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': ''}  
    for s in specials:
        text = text.replace(s, specials[s])
    
    return text

def correct_spelling(x, dic):
    '''Corrects common spelling errors'''   
    for word in dic.keys():
        x = x.replace(word, dic[word])
    return x

def remove_space(text):
    '''Removes  spaces'''   
    #Removes  spaces 
    text = text.strip()
    text = text.split()
    return " ".join(text)

def text_preprocessing_pipeline(text):
    '''Cleaning and parsing the text.'''
    text = clean_text(text)
    text = clean_contractions(text, contraction_mapping)
    text = clean_special_chars(text, punct, punct_mapping)
    text = correct_spelling(text, mispell_dict)
    text = remove_space(text)
    return text

sen = text_preprocessing_pipeline(sen)
z="103;"+sen
z = pd.read_csv(io.StringIO(sen), sep=";")
z = tokenizer1.texts_to_sequences(z)
z = pad_sequences(z, padding='post', maxlen=maxlen)
arr = model_curr.predict(z)
lt=[]
k=0
ind=0
mx=0
for i in arr[0]:
  # print(i)
  if i>0.5:
    lt.append(dct[k])
  if i>mx:
    mx=i
    ind=k
  k=k+1
if len(lt)==0:
  lt.append(dct[ind])
for emo in lt:
  print(emo)

sadness


In [ ]:
# true_sad=0
# false_sad=0
# true_joy=0
# false_joy=0
# true_surp=0
# false_surp=0
# true_ang=0
# false_ang=0
# true_fear=0
# false_fear=0
t_p=np.array([0,0,0,0,0])
f_p=np.array([0,0,0,0,0])
# f_n=t_p
# t_n=t_p
tot = np.array([0,0,0,0,0])

In [ ]:
import io
for i in range(500):
  sen = df_test['Text'][i]
  z="103;"+sen
  z = pd.read_csv(io.StringIO(sen), sep=";")
  z = tokenizer.texts_to_sequences(z)
  z = pad_sequences(z, padding='post', maxlen=maxlen)
  z = X_t[i]
  z=z.reshape(1,40)
  arr = model_curr.predict(z)
  lt=[]
  k=0
  ind=0
  mx=0
  rlt=[]
  for ik in arr[0]:
    if ik>0.5:
      lt.append(k)
    if ik>mx:
      mx=ik
      ind=k
    k=k+1
    if len(lt)==0:
      lt.append(ind)
  if y_t['anger'][i]==1:
    tot[0]=tot[0]+1
    rlt.append(0)
  if y_t['fear'][i]==1:
    tot[1]=tot[1]+1
    rlt.append(1)
  if y_t['joy'][i]==1:
    tot[2]=tot[2]+1
    rlt.append(2)
  if y_t['sadness'][i]==1:
    tot[3]=tot[3]+1
    rlt.append(3)
  if y_t['surprise'][i]==1:
    tot[4]=tot[4]+1
    rlt.append(4)
  # print(df_test['Text'][i])
  # for p in lt:
  #   print(p)
  # for q in rlt:
  #   print(q)
  for xz in rlt:
    if xz in lt:
      t_p[xz]=t_p[xz]+1
  for gz in lt:
    if gz not in rlt:
      f_p[gz]=f_p[gz]+1
      


NameError: ignored

In [ ]:
X_t[0].shape

(40,)

In [ ]:
tot

array([106,  17, 271,  56,  82])

In [ ]:
t_p

array([106,   7, 236,  28,  31])

In [ ]:
f_p

array([394,   3,  51,  13,  18])

In [ ]:
lt=[]

In [ ]:
df_test.shape

(3520, 7)

In [ ]:
k=0
ind=0
mx=0
for i in arr[0]:
  # print(i)
  if i>0.5:
    lt.append(dct[k])
  if i>mx:
    mx=i
    ind=k
  k=k+1


In [ ]:
# len(lt)

In [ ]:
if len(lt)==0:
  lt.append(dct[ind])

In [ ]:
for emo in lt:
  print(emo)

joy


In [ ]:
# eg =pad_sequences(eg, padding='post', maxlen=maxlen)

In [ ]:
# dct = {}
# dct['😀']="good cheer"
# dct['🤣']="so funny"
# dct['😇']="blessings"
# dct['❤️']="love"
# dct['😂'] = "funny"
# dct['😅']="embarrassment"
# dct['😭']="cry"
# dct['🙄']="disapproval"
# dct['😢']="sad"
# dct['👍']="approval"
# dct['🤔']="mocking"
# dct['😔'] = "Saddened by life"
# dct['😁']="happiness"
# dct['😆']="hearty laughter"
# dct['😉'] = "playful"
# dct['😀']="good cheer"
# dct['🤣']="so funny"
# dct['😇']="blessings"
# dct['♥️']="love"
# dct['😎'] = "cool"
# dct['☺']="affection"
# dct['💕']="cry"
# dct['😐']="neutral"
# dct['❤']="love"
# dct['👏']="applause"
# dct['🤷‍♀️']="ignorance"
# dct['🙃'] = "sarcasm"
# dct['‍♀️']="feminine"
# dct['🙏']="gratitude"
# dct['😪']="tired"
# dct['👀']="draw attention"
# dct['🤗']="affection"
# dct['😤'] = "frustration"
# dct['🙂']="friendly sentiments"
# dct['😒']="displeasure"
# dct['🙄']="disapproval"
# dct['👌']="approval"
# dct['💓']="love"
# dct['😞']="disappointment"
# dct['🎂'] = "cake"
# dct['😣']="helplessness"
# dct['😩']="exhaustion"
# dct['🙌🏻'] = "express joy"
# dct['😑']="annoyance "
# dct['🤦🏻‍♀️']="disbelief"
# dct['😝']="playfulness"
# dct['😘']="romantic affection"
# dct['💖'] = "love"
# dct['😳']="shyness"
# dct['🦈']="dangerous"
# dct['😮']="surprise"
# dct['😫']="annoyance"
# dct['🤔']="thinking"
# dct['🤢']="disgust"
# dct['🎶'] = "music"
# dct['‍👍🏼']="approval"
# dct['🤷🏻‍♀️']="shrug"
# dct['♥']="love"
# dct['💗']="love"
# dct['😍']="extreme positivity"
# dct['🤣']="very funny"
# dct['😋'] = "testy"
# dct['🤷‍♂️']="shrug"
# dct['✨']="excitement "
# dct['😖']="so confused"
# dct['😇']="blessings"
# dct['💙']="love"
# dct['😟']="moderately sad"
# dct['🍻'] = "Cheers"
# dct['🌿']="nature"
# dct['💪🏻']="power"
# dct['😜'] = "sense of fun"
# dct['🤠']="agile"
# dct['💩']="crap"
# dct['🤕']="emotional hurting"
# dct['👑']="crown"
# dct['🤦'] = "frustration"
# dct['✊🏼']="solidarity"
# dct['🤯']="disbelief"
# dct['🥺']="sympathy"
# dct['🤪']="joking"
# dct['🤓']="being smart"
# dct['🎉']="congratulations"
# dct['🤩'] = "affection"
# dct['👊']="agree"
# dct['💭']="trying to make a decision"
# dct['🍰']="cake"
# dct['🙏🏻']="gratitude"
# dct['🤮']="disgust"
# dct['👩'] = "woman"
# dct['💄']="kisses"
# dct['💲']="money"
# dct['😏']="flirting"
# dct['😹']="joy"
# dct['💓']="love"
# dct['😊']="affection"
# dct['❣️'] = "love"
# dct['💜']="love"
# dct['🤗']="affection"
# dct['😻'] = "love"
# dct['🤞']="luck"
# dct['🥀']="heartbreak"
# dct['👌🏽']="approval"
# dct['🔥']="fire"
# dct['😡'] = "angry"
# dct['🤢']="disgust"
# dct['😰']="fear"
# dct['🙌🏿']="express joy"
# dct['💰']="money"
# dct['🙋']="asking a favour"
# dct['😠']="angry"
# dct['😓'] = "frustration"
# dct['🙊']="secrecy"
# dct['🥰']="affection"
# dct['🍿']="movie"
# dct['😧']="shock"
# dct['👎']="disapproval"
# dct['🙏🏽']="gratitude"
# dct['💛'] = "love"
# dct['💔']="heartbreak"
# dct['👊']="agree"
# dct['💁']="unsure"
# dct['✌']="peace"
# dct['😱']="extreme"
# dct['🙆']="ok"
# dct['🖕'] = "offensive"
# dct['🎣']="fishing"
# dct['🍀']="good luck"

In [ ]:
# df['new_text'] = df['Text'].apply(fun)

In [ ]:
# df['Text']